In [1]:
import sys
sys.path.append("/user/al4263/rlhf/Reward_Modeling")
# change it to you home dir
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import utils
from models.reward_enn import RewardENN, RewardENNConfig
from models.vanilla_reward import VanillaReward, VanillaRewardConfig
import torch
import numpy as np
import matplotlib.pyplot as plt

/user/al4263/.conda/envs/rlhf_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-24 15:30:01,098] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
model_dir = "/shared/share_mala/leon/reward-enn/alpaca_human_preference/lr1e-05-gradient_acc1-train_batch_size4-ref_size10-enn_dim64-num_ref_train10-weight_decay0.01"
backbone_model_dir = "/shared/share_mala/leon/llama-3b-sft"
tokenizer = AutoTokenizer.from_pretrained('/shared/share_mala/leon/llama-3b-sft')
#AutoConfig.register("reward_enn", RewardENNConfig)

config = RewardENNConfig(
    backbone_model_name_or_path=backbone_model_dir,
    ref_size=10,
    enn_hidden_size=64,
    enn_output_size=1,
    enn_gain=1,
    lmbda=1,
    )

model = RewardENN.from_pretrained(
    model_dir,
    flash_attn=False,
    fp16=False,
    bf16=True,
    config=config,
    )

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/shared/share_mala/leon/reward-enn/alpaca_human_preference/lr1e-05-gradient_acc1-train_batch_size4-ref_size10-enn_dim64-num_ref_train10-weight_decay0.01'. Use `repo_type` argument if needed.

In [3]:
model = model.eval()
model.to(torch.bfloat16)

RewardENN(
  (backbone_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32001, 3200, padding_idx=0)
      (layers): ModuleList(
        (0-25): 26 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
            (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
            (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
            (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
            (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
            (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attenti

In [8]:
# print model.eta_net parameters in for loop
for name, param in model.eta_net.named_parameters():
    print(name, param)

model.0.weight Parameter containing:
tensor([[ 3.5889e-02, -1.8311e-02,  3.7842e-02,  ..., -3.2959e-02,
         -3.8574e-02,  9.7046e-03],
        [-3.7842e-02, -2.0630e-02,  3.2471e-02,  ..., -1.8799e-02,
         -3.4668e-02, -4.1748e-02],
        [-3.5645e-02, -1.1536e-02, -2.2217e-02,  ...,  2.6489e-02,
          4.3640e-03,  2.8076e-02],
        ...,
        [ 2.2949e-02, -3.8818e-02, -3.3203e-02,  ...,  4.5300e-05,
         -4.4556e-03, -3.1494e-02],
        [ 3.9307e-02, -2.7832e-02,  3.4027e-03,  ..., -2.6245e-02,
          2.8198e-02,  3.9795e-02],
        [-1.6113e-02, -2.8809e-02,  4.0771e-02,  ...,  2.5635e-02,
         -2.9663e-02, -2.8931e-02]], dtype=torch.bfloat16, requires_grad=True)
model.0.bias Parameter containing:
tensor([ 7.6771e-05, -8.6784e-05,  8.0109e-05, -7.2002e-05,  1.4305e-05,
        -8.0585e-05, -4.7922e-05, -2.8968e-05, -1.0490e-04, -5.5313e-05,
         3.3081e-06,  7.1228e-06, -8.0466e-06, -5.7220e-05,  6.1989e-06,
         9.9182e-05, -1.8239e-05,  

In [34]:
batch_size = 10
x_data = torch.randn(10 , 3200)
z = torch.randn(5, 10)
x_data_repeat = x_data.unsqueeze(1).repeat(1, 5, 1)
x_data_repeat = x_data_repeat.to(torch.bfloat16)
z = z.to(torch.bfloat16)

output = model.eta_net(x = x_data_repeat, z=z, return_full_z=False)

output

tensor([[ 0.0928],
        [ 0.3691],
        [-0.0977],
        [ 2.2344],
        [ 1.1719],
        [-0.1465],
        [ 2.1562],
        [-0.0957],
        [ 1.9844],
        [ 0.6328]], dtype=torch.bfloat16, grad_fn=<MulBackward0>)

In [9]:
import json
import os
from data.data_loader import pairwise_data_tokenized, PairwiseDyadicAugmentedTokenizedData
data_dir = "/user/al4263/rlhf/Reward_Modeling/data/dataset/alpaca_human_preference"
eval_dir = data_dir + "/eval.json"
joint_eval_dir = data_dir + "/joint_eval.json"

with open(eval_dir) as f:
    eval_raw_data = json.load(f)
with open(joint_eval_dir) as f:
    joint_eval_raw_data = json.load(f)

model_max_length = 512
ref_size = 10
num_ref_joint_eval = 100
project = "reward-enn"

# round down to the nearst multiple of 8: 
eval_dataset = pairwise_data_tokenized(eval_raw_data, tokenizer, model_max_length)
joint_eval_dataset = PairwiseDyadicAugmentedTokenizedData(joint_eval_raw_data, tokenizer, model_max_length, ref_size, num_ref_joint_eval, project)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=1, shuffle= False)
joint_eval_dataloader = torch.utils.data.DataLoader(joint_eval_dataset, batch_size=1, shuffle= False)


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
num_samples = 100000
ref_size = 10

In [14]:
model = model.to(device)    

In [27]:
model_out_diff = []

for _, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        p_1, p_2, p_1_att, p_2_att, label = batch
        p_1 = p_1.to(device)
        p_2 = p_2.to(device)
        p_1_att = p_1_att.to(device)
        p_2_att = p_2_att.to(device)
        label = label.to(device)
        # change the mean of z_samples to 10

        z_samples = (torch.randn(num_samples, ref_size)).to(device)
        z_samples = z_samples.to(torch.bfloat16)
        reward_1 = model(p_1, p_1_att, z_samples, return_full_z = True)
        reward_2 = model(p_2, p_2_att, z_samples, return_full_z = True)
        #print('model_out 1', reward_1.model_out)
        #print('model_out 2', reward_2.model_out)
        print('model_out_diff', reward_1.model_out - reward_2.model_out)
        print('eta_diff', reward_1.eta_list.mean() - reward_2.eta_list.mean())
        print('p_diff', reward_1.p_list.mean() - reward_2.p_list.mean())
        print('reward_diff', reward_1.reward_list.mean() - reward_2.reward_list.mean())
        print('-----------------------------------------')
        #print('eta_out 1', reward_1.eta_list.mean())
        #print('eta_out 2', reward_2.eta_list.mean())
        #print('p_out 1', reward_1.p_list.mean())    
        #print('p_out 2', reward_2.p_list.mean())





model_out_diff tensor([[0.0469]], device='cuda:0', dtype=torch.bfloat16)
eta_diff tensor(0.0032, device='cuda:0', dtype=torch.bfloat16)
p_diff tensor(-0.0002, device='cuda:0', dtype=torch.bfloat16)
reward_diff tensor(0.0488, device='cuda:0', dtype=torch.bfloat16)
-----------------------------------------
model_out_diff tensor([[-1.5234]], device='cuda:0', dtype=torch.bfloat16)
eta_diff tensor(-0.1143, device='cuda:0', dtype=torch.bfloat16)
p_diff tensor(-0.0325, device='cuda:0', dtype=torch.bfloat16)
reward_diff tensor(-1.6719, device='cuda:0', dtype=torch.bfloat16)
-----------------------------------------
model_out_diff tensor([[-0.0801]], device='cuda:0', dtype=torch.bfloat16)
eta_diff tensor(0.0029, device='cuda:0', dtype=torch.bfloat16)
p_diff tensor(-0.0055, device='cuda:0', dtype=torch.bfloat16)
reward_diff tensor(-0.0840, device='cuda:0', dtype=torch.bfloat16)
-----------------------------------------
model_out_diff tensor([[0.6758]], device='cuda:0', dtype=torch.bfloat16)
eta_

KeyboardInterrupt: 

In [ ]:
# create random tensor of this shape (batch_size, num_Z_size, self.ref_size, self.output_size)
batch_size = 1
num_Z_size = 100
ref_size = 10
output_size = 1
x_tilda_og = torch.randn(batch_size, num_Z_size, ref_size, output_size)

x_tilda_list = []
for i in range(500):
    z = torch.randn(num_Z_size, ref_size)
    z = z.unsqueeze(0).repeat(x_tilda_og.shape[0], 1, 1)
    x_tilda = torch.transpose(x_tilda_og, 2, 3) @ z.unsqueeze(3)
    x_tilda = torch.mean(x_tilda, dim=1)
    x_tilda = x_tilda.view(-1, 1)
    x_tilda_list.append(x_tilda.squeeze(1))


data = x_tilda_list

# Convert to numpy
data_np = np.array(data)

# Plot histogram
plt.hist(data_np, bins=100, density=True, alpha=0.6, color='g')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram')
plt.grid(True)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Generate data
data = x_tilda_list

# Convert to numpy
data_np = np.array(data)

# Plot histogram
plt.hist(data_np, bins=100, density=True, alpha=0.6, color='g')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram')
plt.grid(True)
plt.show()


In [ ]:
data_np.shape